In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TitanicSpark").getOrCreate()
spark

25/08/23 08:02:37 WARN Utils: Your hostname, DESKTOP-VJA6A5N resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/23 08:02:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/23 08:02:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/23 08:02:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/23 08:02:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:

import os
os.getcwd()
data_ingestion = os.chdir('../artifacts/data_ingestion')
print(os.getcwd())

/home/amit/python/Industrial_AI_project/IITM_MLops_titanic_dataset_github_clone/IITM-MLProject-kaggle-Titanic-dataset/artifacts/data_ingestion


In [4]:
titanic_df = spark.read.csv(f'{os.getcwd()}/train.csv', header=True, inferSchema=True)
titanic_df.printSchema()
titanic_df.show(5)

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|   

In [5]:
from pyspark.sql.functions import col, sum

# Check for null values in each column
missing_values = titanic_df.select(
    [sum(col(c).isNull().cast("integer")).alias(c) for c in titanic_df.columns]
)

missing_values.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [15]:
# show missing values in titanic_df
titanic_df.select([sum(col(c).isNull().cast("integer")).alias(c) for c in titanic_df.columns]).show()
titanic_df.describe().show()

+-----------+--------+------+----+---+---+-----+-----+------+----+--------+----------+-------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Embarked|FamilySize|IsAlone|
+-----------+--------+------+----+---+---+-----+-----+------+----+--------+----------+-------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|       0|         0|      0|
+-----------+--------+------+----+---+---+-----+-----+------+----+--------+----------+-------+



25/08/23 08:06:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+--------+-----------------+------------------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Embarked|       FamilySize|           IsAlone|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+--------+-----------------+------------------+
|  count|              891|                891|               891|                 891|   891|               891|               891|                891|               891|              891|     891|              891|               891|
|   mean|            446.0| 0.3838383838383838| 2.308641

25/08/24 04:24:02 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 500524 ms exceeds timeout 120000 ms
25/08/24 04:24:02 WARN SparkContext: Killing executors is not supported by current scheduler.
25/08/24 04:24:06 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1240)
	at o

In [13]:
from pyspark.sql.functions import mean, mode

# Impute missing 'Age' values with the mean age
mean_age = titanic_df.select(mean("Age")).collect()[0][0]
titanic_df = titanic_df.na.fill({"Age": mean_age})

# Impute missing 'Embarked' values with the mode (most frequent value)
# Calculate mode for Embarked
mode_embarked = titanic_df.groupBy("Embarked").count().orderBy("count", ascending=False).collect()[0][0]
titanic_df = titanic_df.na.fill({"Embarked": mode_embarked})

# Drop the 'Cabin' column due to a large number of missing values
titanic_df = titanic_df.drop("Cabin")

# Verify that missing values have been addressed in the targeted columns
missing_values_after_imputation = titanic_df.select(
    [sum(col(c).isNull().cast("integer")).alias(c) for c in titanic_df.columns]
)
missing_values_after_imputation.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+--------+----------+-------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Embarked|FamilySize|IsAlone|
+-----------+--------+------+----+---+---+-----+-----+------+----+--------+----------+-------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|       0|         0|      0|
+-----------+--------+------+----+---+---+-----+-----+------+----+--------+----------+-------+



In [9]:
# feature engineering

from pyspark.sql.functions import col, when

# Create 'FamilySize' feature
titanic_df = titanic_df.withColumn("FamilySize", col("SibSp") + col("Parch") + 1)

# Create 'IsAlone' feature
titanic_df = titanic_df.withColumn("IsAlone", when(col("FamilySize") == 1, 1).otherwise(0))

# Display the schema to confirm the new columns
titanic_df.printSchema()

# Show the first few rows with the new columns
titanic_df.select("SibSp", "Parch", "FamilySize", "IsAlone", "PassengerId").show(5)

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- FamilySize: integer (nullable = true)
 |-- IsAlone: integer (nullable = false)

+-----+-----+----------+-------+-----------+
|SibSp|Parch|FamilySize|IsAlone|PassengerId|
+-----+-----+----------+-------+-----------+
|    1|    0|         2|      0|          1|
|    1|    0|         2|      0|          2|
|    0|    0|         1|      1|          3|
|    1|    0|         2|      0|          4|
|    0|    0|         1|      1|          5|
+-----+-----+----------+-------+-----------+
only showing top 5 rows



In [10]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Identify categorical columns
categorical_cols = ["Sex", "Embarked"]
indexed_cols = [col + "_indexed" for col in categorical_cols]
encoded_cols = [col + "_encoded" for col in categorical_cols]

# Create StringIndexers for categorical columns
indexers = [StringIndexer(inputCol=col, outputCol=col + "_indexed", handleInvalid="keep") for col in categorical_cols]

# Chain indexers and apply to the DataFrame
from pyspark.ml import Pipeline
indexer_pipeline = Pipeline(stages=indexers)
titanic_indexed = indexer_pipeline.fit(titanic_df).transform(titanic_df)

# Create OneHotEncoders for the indexed columns
encoders = [OneHotEncoder(inputCol=col, outputCol=col + "_encoded") for col in indexed_cols]

# Chain encoders and apply to the DataFrame
encoder_pipeline = Pipeline(stages=encoders)
titanic_encoded = encoder_pipeline.fit(titanic_indexed).transform(titanic_indexed)

# Drop original categorical and intermediate indexed columns
columns_to_drop = categorical_cols + indexed_cols
titanic_final = titanic_encoded.drop(*columns_to_drop)

# Display the schema of the DataFrame after one-hot encoding
titanic_final.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- FamilySize: integer (nullable = true)
 |-- IsAlone: integer (nullable = false)
 |-- Sex_indexed_encoded: vector (nullable = true)
 |-- Embarked_indexed_encoded: vector (nullable = true)



In [11]:
# Assemble features
from pyspark.ml.feature import VectorAssembler

# Define the list of feature columns (numerical and one-hot encoded)
feature_columns = [
    'Pclass',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'FamilySize',
    'IsAlone',
    'Sex_indexed_encoded',
    'Embarked_indexed_encoded'
]

# Instantiate VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

# Transform the DataFrame to include the features vector
titanic_final = assembler.transform(titanic_final)

# Show the schema to confirm the new 'features' column
titanic_final.printSchema()

# Display the first few rows showing the 'features' column
titanic_final.select('features', 'Survived').show(5, truncate=False)

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- FamilySize: integer (nullable = true)
 |-- IsAlone: integer (nullable = false)
 |-- Sex_indexed_encoded: vector (nullable = true)
 |-- Embarked_indexed_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)

+------------------------------------------------------+--------+
|features                                              |Survived|
+------------------------------------------------------+--------+
|[3.0,22.0,1.0,0.0,7.25,2.0,0.0,1.0,0.0,1.0,0.0,0.0]   |0       |
|[1.0,38.0,1.0,0.0,71.2833,2.0,0.0,0.0,1.0,0.0,1.0,0.0]|1       |
|[3.0,26.0,0.0,0.0,7.925,1.0,1.0,0.0,1.0,1.0,0.0,0.0]  |1       |
|[1.0,35.0,1.0,0.0,53

In [12]:
# Display the schema of the titanic_final DataFrame
titanic_final.printSchema()

# Show the first 10 rows with selected columns
titanic_final.select(
    'Survived',
    'Pclass',
    'Age',
    'FamilySize',
    'IsAlone',
    'Sex_indexed_encoded',
    'Embarked_indexed_encoded',
    'features'
).show(10, truncate=False)

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- FamilySize: integer (nullable = true)
 |-- IsAlone: integer (nullable = false)
 |-- Sex_indexed_encoded: vector (nullable = true)
 |-- Embarked_indexed_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)

+--------+------+-----------------+----------+-------+-------------------+------------------------+------------------------------------------------------------------+
|Survived|Pclass|Age              |FamilySize|IsAlone|Sex_indexed_encoded|Embarked_indexed_encoded|features                                                          |
+--------+------+-----------------+----------+-------+-------------------+---------

## Summary:

### Data Analysis Key Findings

*   The initial attempt to load "train.csv" and "test.csv" failed due to incorrect file paths. A single "titanic.csv" file was successfully downloaded and used instead.
*   Missing values were identified in the 'Age' (177), 'Cabin' (687), and 'Embarked' (2) columns.
*   Missing 'Age' values were imputed with the mean age, and missing 'Embarked' values were imputed with the mode.
*   The 'Cabin' column was dropped due to a high number of missing values.
*   Two new features, 'FamilySize' (calculated as SibSp + Parch + 1) and 'IsAlone' (1 if FamilySize is 1, 0 otherwise), were successfully created.
*   Categorical features 'Sex' and 'Embarked' were successfully one-hot encoded using StringIndexer and OneHotEncoder.
*   All relevant features (numerical, engineered, and one-hot encoded) were successfully assembled into a single vector column named 'features'.

### Insights or Next Steps

*   The processed data is now in a suitable format (a single feature vector column) for training a machine learning model using SparkML.
*   The next logical step is to split the data into training and testing sets and then train a classification model (e.g., Logistic Regression or RandomForest) on the 'features' column to predict the 'Survived' target variable.


In [41]:
import os

In [52]:
%pwd

'/home/amit/python/Industrial_AI_project/IITM_MLops_titanic_dataset_github_clone/IITM-MLProject-kaggle-Titanic-dataset'

In [43]:
os.chdir('../')
%pwd

'/home/amit/python/Industrial_AI_project/IITM_MLops_titanic_dataset_github_clone/IITM-MLProject-kaggle-Titanic-dataset'

In [53]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    input_data_file: Path
    local_data_file: Path

In [54]:
# from Titanic_dataset_analysis import constants as c
# from Titanic_dataset_analysis.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
from Titanic_dataset_analysis.utils.common import read_yaml, create_directories

In [55]:
CONFIG_FILE_PATH

PosixPath('config/config.yaml')

In [56]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        print(os.getcwd())
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        config = self.config.data_preprocessing

        create_directories([config.root_dir])

        data_preprocessing_config = DataPreprocessingConfig(
            root_dir=config.root_dir,
            input_data_file=config.input_data_file,
            local_data_file=config.local_data_file
        )

        return data_preprocessing_config

In [57]:
import os
import pandas as pd
from Titanic_dataset_analysis import logger
from Titanic_dataset_analysis.utils.common import get_size

In [58]:
class DataPreprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config


    
    def read_file(self):
        if not os.path.exists(self.config.input_data_file):
            logger.info(f"File download failed in previous step! Please check the location mentioned : {self.config.input_data_file}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.input_data_file))}")  

        self.df = pd.read_csv(self.config.input_data_file)
        logger.info(f"Input file read from {self.config.input_data_file}")
    
    def preprocess_and_save_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        # Basic cleaning / feature engineering
        mean_age = self.df['Age'].mean()
        mode_embarked = self.df['Embarked'].mode().iloc[0] if 'Embarked' in self.df.columns else 'S'
        self.df['Age'] = pd.to_numeric(self.df['Age'], errors='coerce').fillna(mean_age)
        if 'Fare' in self.df.columns:
            self.df['Fare'] = pd.to_numeric(self.df['Fare'], errors='coerce').fillna(0.0)
        self.df['SibSp'] = pd.to_numeric(self.df.get('SibSp', 0), errors='coerce').fillna(0).astype(int)
        self.df['Parch'] = pd.to_numeric(self.df.get('Parch', 0), errors='coerce').fillna(0).astype(int)
        self.df['FamilySize'] = self.df['SibSp'] + self.df['Parch'] + 1
        self.df['IsAlone'] = (self.df['FamilySize'] == 1).astype(int)
        if 'Embarked' in self.df.columns:
            self.df['Embarked'] = self.df['Embarked'].fillna(mode_embarked)
        
        os.makedirs(self.config.root_dir, exist_ok=True)
        self.df.to_csv(self.config.local_data_file, index=False)
        logger.info(f"Data Preprocessing done successfully.")
            
        # unzip_path = self.config.unzip_dir
        # os.makedirs(unzip_path, exist_ok=True)
        # with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
        #     zip_ref.extractall(unzip_path)

In [59]:
try:
    config = ConfigurationManager()
    data_preprocessing_config = config.get_data_preprocessing_config()
    data_preprocessing = DataPreprocessing(config=data_preprocessing_config)
    data_preprocessing.read_file()
    data_preprocessing.preprocess_and_save_file()
except Exception as e:
    raise e

/home/amit/python/Industrial_AI_project/IITM_MLops_titanic_dataset_github_clone/IITM-MLProject-kaggle-Titanic-dataset
